### GraphSAGE -- GNN范轴 -- Embedding
- 是一种能够利用顶点的属性信息高效产生未知顶点embedding的一种归纳式(inductive)学习的框架
- 只利用节点之间的连接关系 -- 对点进行编码

In [1]:
import sys
import os
import argparse
import pyhocon
import random
from collections import defaultdict
import numpy as np
import math

from sklearn.utils import shuffle
from sklearn.metrics import f1_score

import torch
import torch.nn as nn
import torch.nn.functional as F

### 参数设置

In [2]:
parser = argparse.ArgumentParser(description='pytorch version of GraphSAGE')

parser.add_argument('--dataSet', type=str, default='cora')
parser.add_argument('--agg_func', type=str, default='MEAN')
parser.add_argument('--epochs', type=int, default=2)
parser.add_argument('--b_sz', type=int, default=20)
parser.add_argument('--seed', type=int, default=824)
parser.add_argument('--gcn', action='store_true')
parser.add_argument('--learn_method', type=str, default='sup')
parser.add_argument('--unsup_loss', type=str, default='normal')
parser.add_argument('--max_vali_f1', type=float, default=0)
parser.add_argument('--name', type=str, default='debug')

# attention
parser.add_argument('--config', type=str, default=r'C:\Users\sss\Desktop/experiments.conf')  #/src

args = parser.parse_args(args=[])

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed_all(args.seed)

In [5]:
config = pyhocon.ConfigFactory.parse_file(args.config)

In [6]:
config

ConfigTree([('file_path',
             ConfigTree([('workdir', '../'),
                         ('cora_content',
                          'C:\\Users\\sss\\Desktop\\cora/cora.content'),
                         ('cora_cite',
                          'C:\\Users\\sss\\Desktop\\cora/cora.cites'),
                         ('pubmed_paper',
                          '../pubmed-data/Pubmed-Diabetes.NODE.paper.tab'),
                         ('pubmed_cites',
                          '../pubmed-data/Pubmed-Diabetes.DIRECTED.cites.tab')])),
            ('setting',
             ConfigTree([('num_layers', 2), ('hidden_emb_size', 128)]))])

### 数据集

In [7]:
class DataCenter(object):
    """docstring for DataCenter"""
    def __init__(self, config):
        super(DataCenter, self).__init__()
        self.config = config
        
    def load_dataSet(self, dataSet='cora'):
        if dataSet == 'cora':
            cora_content_file = self.config['file_path.cora_content']
            cora_cite_file = self.config['file_path.cora_cite']

            feat_data = []
            labels = [] # label sequence of node
            node_map = {} # map node to Node_ID
            label_map = {} # map label to Label_ID
            with open(cora_content_file) as fp:
                for i, line in enumerate(fp):
                    info = line.strip().split()
                    feat_data.append([float(x) for x in info[1: -1]])
                    node_map[info[0]] = i  # 节点的index
                    if not info[-1] in label_map:
                        label_map[info[-1]] = len(label_map)
                    labels.append(label_map[info[-1]])
            feat_data = np.asarray(feat_data)
            labels = np.asarray(labels, dtype=np.int64)
                
            adj_lists = defaultdict(set)
            with open(cora_cite_file) as fp:
                for i, line in enumerate(fp):
                    info = line.strip().split()
                    assert len(info) == 2
                    paper1 = node_map[info[0]]
                    paper2 = node_map[info[1]]
                    adj_lists[paper1].add(paper2)
                    adj_lists[paper2].add(paper1)

            assert len(feat_data) == len(labels) == len(adj_lists)
            test_indexs, val_indexs, train_indexs = self._split_data(feat_data.shape[0])  # 选择data中训练，验证，测试集

            setattr(self, dataSet + '_test', test_indexs)
            setattr(self, dataSet + '_val', val_indexs)
            setattr(self, dataSet + '_train', train_indexs)

            setattr(self, dataSet + '_feats', feat_data)
            setattr(self, dataSet + '_labels', labels)
            setattr(self, dataSet + '_adj_lists', adj_lists)

        elif dataSet == 'pubmed':
            pubmed_content_file = self.config['file_path.pubmed_paper']
            pubmed_cite_file = self.config['file_path.pubmed_cites']

            feat_data = []
            labels = [] # label sequence of node
            node_map = {} # map node to Node_ID
            with open(pubmed_content_file) as fp:
                fp.readline()
                feat_map = {entry.split(":")[1]:i-1 for i,entry in enumerate(fp.readline().split("\t"))}
                for i, line in enumerate(fp):
                    info = line.split("\t")
                    node_map[info[0]] = i
                    labels.append(int(info[1].split("=")[1])-1)
                    tmp_list = np.zeros(len(feat_map)-2)
                    for word_info in info[2:-1]:
                        word_info = word_info.split("=")
                        tmp_list[feat_map[word_info[0]]] = float(word_info[1])
                    feat_data.append(tmp_list)
                
            feat_data = np.asarray(feat_data)
            labels = np.asarray(labels, dtype=np.int64)
                
            adj_lists = defaultdict(set)
            with open(pubmed_cite_file) as fp:
                fp.readline()
                fp.readline()
                for line in fp:
                    info = line.strip().split("\t")
                    paper1 = node_map[info[1].split(":")[1]]
                    paper2 = node_map[info[-1].split(":")[1]]
                    adj_lists[paper1].add(paper2)
                    adj_lists[paper2].add(paper1)
                
            assert len(feat_data) == len(labels) == len(adj_lists)
            test_indexs, val_indexs, train_indexs = self._split_data(feat_data.shape[0])

            setattr(self, dataSet + '_test', test_indexs)
            setattr(self, dataSet + '_val', val_indexs)
            setattr(self, dataSet + '_train', train_indexs)

            setattr(self, dataSet + '_feats', feat_data)
            setattr(self, dataSet + '_labels', labels)
            setattr(self, dataSet + '_adj_lists', adj_lists)


    def _split_data(self, num_nodes, test_split = 3, val_split = 6):
        rand_indices = np.random.permutation(num_nodes)

        test_size = num_nodes // test_split
        val_size = num_nodes // val_split
        train_size = num_nodes - (test_size + val_size)

        test_indexs = rand_indices[:test_size]
        val_indexs = rand_indices[test_size:(test_size+val_size)]
        train_indexs = rand_indices[(test_size+val_size):]
            
        return test_indexs, val_indexs, train_indexs

In [8]:
# load data
ds = args.dataSet
dataCenter = DataCenter(config)
dataCenter.load_dataSet(ds)  # 读取数据

features = torch.FloatTensor(getattr(dataCenter, ds + '_feats')).to(device)
labels = torch.FloatTensor(getattr(dataCenter, ds + '_labels')).to(device)
adj = getattr(dataCenter, ds + '_adj_lists')

In [9]:
[features.shape, labels.shape, len(adj)]

[torch.Size([2708, 1433]), torch.Size([2708]), 2708]

In [10]:
features

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')

### 模型一：GraphSAGE
    第一层：[2708, 1433] -> [2708, 128]
    第二层：[2708, 128] -> [2708, 128]

In [11]:
class SageLayer(nn.Module):
    """
    Encodes a node's using 'convolutional' GraphSage approach
    """
    def __init__(self, input_size, out_size, gcn=False): 
        super(SageLayer, self).__init__()

        self.input_size = input_size
        self.out_size = out_size
        self.gcn = gcn
        self.weight = nn.Parameter(torch.FloatTensor(out_size, self.input_size if self.gcn else 2 * self.input_size)) # 创建weight

        self.init_params()  # 初始化参数

    def init_params(self):
        for param in self.parameters():
            nn.init.xavier_uniform_(param)

    def forward(self, self_feats, aggregate_feats, neighs=None):
        """
        Generates embeddings for a batch of nodes
        nodes -- list of nodes
        """
        if not self.gcn:
            combined = torch.cat([self_feats, aggregate_feats], dim=1)   # concat自己信息和邻居信息
        else:
            combined = aggregate_feats
        combined = F.relu(self.weight.mm(combined.t())).t()
        return combined

In [12]:
class GraphSage(nn.Module):
    """docstring for GraphSage"""
    def __init__(self, num_layers, input_size, out_size, raw_features, adj_lists, device, gcn=False, agg_func='MEAN'):
        super(GraphSage, self).__init__()

        self.input_size = input_size
        self.out_size = out_size
        self.num_layers = num_layers
        self.gcn = gcn
        self.device = device
        self.agg_func = agg_func

        self.raw_features = raw_features  # 点的特征
        self.adj_lists = adj_lists  # 边的连接

        for index in range(1, num_layers + 1):
            layer_size = out_size if index != 1 else input_size
            setattr(self, 'sage_layer' + str(index), SageLayer(layer_size, out_size, gcn=self.gcn))

    def forward(self, nodes_batch):
        """
        Generates embeddings for a batch of nodes.
        nodes_batch -- batch of nodes to learn the embeddings.    《minbatch 过程，涉及到的所有节点》
        """
        lower_layer_nodes = list(nodes_batch)
        nodes_batch_layers = [(lower_layer_nodes,)]  # 第一次放入的节点，batch节点
        # self.dc.logger.info('get_unique_neighs.')
        for i in range(self.num_layers):  # 每层的Sage
            lower_samp_neighs, lower_layer_nodes_dict, lower_layer_nodes = self._get_unique_neighs_list(lower_layer_nodes)  # 获得neighbors。 聚合自己和邻居节点，点的dict，涉及到的所有节点
            nodes_batch_layers.insert(0, (lower_layer_nodes, lower_samp_neighs, lower_layer_nodes_dict))  # 聚合自己和邻居节点，点的dict，涉及到的所有节点
            # insert,0 从最外层开始聚合
        assert len(nodes_batch_layers) == self.num_layers + 1

        pre_hidden_embs = self.raw_features
        
        for index in range(1, self.num_layers + 1):
            nb = nodes_batch_layers[index][0]   # 聚合自己和周围的节点
            pre_neighs = nodes_batch_layers[index - 1]  # 这层节点的上层邻居的所有信息。聚合自己和邻居节点，点的dict，涉及到的所有节点
            # self.dc.logger.info('aggregate_feats.') aggrefate_feats=>输出GraphSAGE聚合后的信息
            aggregate_feats = self.aggregate(nb, pre_hidden_embs, pre_neighs)  # 聚合函数。nb-这一层的节点， pre_hidden_embs-feature，pre_neighs-上一层节点
            sage_layer = getattr(self, 'sage_layer' + str(index))
            if index > 1:
                nb = self._nodes_map(nb, pre_hidden_embs, pre_neighs)   # 第一层的batch节点，没有进行转换
            # self.dc.logger.info('sage_layer.')
            cur_hidden_embs = sage_layer(
                self_feats=pre_hidden_embs[nb],
                aggregate_feats=aggregate_feats
            )  # 进入SageLayer。weight*concat(node,neighbors)
            pre_hidden_embs = cur_hidden_embs

        return pre_hidden_embs

    def _nodes_map(self, nodes, hidden_embs, neighs):
        layer_nodes, samp_neighs, layer_nodes_dict = neighs
        assert len(samp_neighs) == len(nodes)
        index = [layer_nodes_dict[x] for x in nodes]  # 记录将上一层的节点编号。
        return index

    def _get_unique_neighs_list(self, nodes, num_sample=10):
        _set = set
        to_neighs = [self.adj_lists[int(node)] for node in nodes]    # self.adj_lists边矩阵，获取节点的邻居
        if not num_sample is None:  # 对邻居节点进行采样，如果大于邻居数据，则进行采样
            _sample = random.sample
            samp_neighs = [_set(_sample(to_neigh, num_sample)) if len(to_neigh) >= num_sample else to_neigh for to_neigh in to_neighs]
        else:
            samp_neighs = to_neighs
        samp_neighs = [samp_neigh | set([nodes[i]]) for i, samp_neigh in enumerate(samp_neighs)]  # 聚合本身节点和邻居节点
        _unique_nodes_list = list(set.union(*samp_neighs))  # 这个batch涉及到的所有节点
        i = list(range(len(_unique_nodes_list)))
        unique_nodes = dict(list(zip(_unique_nodes_list, i)))  # 字典编号
        return samp_neighs, unique_nodes, _unique_nodes_list   # 聚合自己和邻居节点，点的dict，涉及到的所有节点

    def aggregate(self, nodes, pre_hidden_embs, pre_neighs, num_sample=10):
        unique_nodes_list, samp_neighs, unique_nodes = pre_neighs   # 聚合自己和邻居节点，涉及到的所有节点，点的dict

        assert len(nodes) == len(samp_neighs)
        indicator = [(nodes[i] in samp_neighs[i]) for i in range(len(samp_neighs))]  # 都是True，因为上文中，将nodes加入到neighs中了
        assert (False not in indicator)
        if not self.gcn:
            samp_neighs = [(samp_neighs[i]-set([nodes[i]])) for i in range(len(samp_neighs))]  # 在把中心节点去掉
        # self.dc.logger.info('2')
        if len(pre_hidden_embs) == len(unique_nodes):  # 如果涉及到所有节点，保留原矩阵。如果不涉及所有节点，保留部分矩阵。
            embed_matrix = pre_hidden_embs
        else:
            embed_matrix = pre_hidden_embs[torch.LongTensor(unique_nodes_list)]
        # self.dc.logger.info('3')  将对应到的边，构建邻接矩阵
        mask = torch.zeros(len(samp_neighs), len(unique_nodes))   # 本层节点数量，涉及到上层节点数量
        column_indices = [unique_nodes[n] for samp_neigh in samp_neighs for n in samp_neigh]  # 构建邻接矩阵
        row_indices = [i for i in range(len(samp_neighs)) for j in range(len(samp_neighs[i]))]
        mask[row_indices, column_indices] = 1   # 加上上两个步骤，都是构建邻接矩阵;
        # self.dc.logger.info('4')
        # mask - 邻接矩阵
        if self.agg_func == 'MEAN':
            num_neigh = mask.sum(1, keepdim=True)    # 按行求和，保持和输入一个维度
            mask = mask.div(num_neigh).to(embed_matrix.device)  # 归一化操作
            aggregate_feats = mask.mm(embed_matrix)   # 矩阵相乘，相当于聚合周围邻接信息求和

        elif self.agg_func == 'MAX':
            # print(mask)
            indexs = [x.nonzero() for x in mask==1]
            aggregate_feats = []
            # self.dc.logger.info('5')
            for feat in [embed_matrix[x.squeeze()] for x in indexs]:
                if len(feat.size()) == 1:
                    aggregate_feats.append(feat.view(1, -1))
                else:
                    aggregate_feats.append(torch.max(feat,0)[0].view(1, -1))
            aggregate_feats = torch.cat(aggregate_feats, 0)

        # self.dc.logger.info('6')

        return aggregate_feats

### 模型二：全连接层
    第三层：[2708, 128] -> [2708, 7]

In [13]:
class Classification(nn.Module):
    def __init__(self, emb_size, num_classes):
        super(Classification, self).__init__()

        #self.weight = nn.Parameter(torch.FloatTensor(emb_size, num_classes)) 最终的输出 (128, num_classes)
        self.layer = nn.Sequential(
            nn.Linear(emb_size, num_classes)
            #nn.ReLU()
        )
        self.init_params()

    def init_params(self):
        for param in self.parameters():
            if len(param.size()) == 2:
                nn.init.xavier_uniform_(param)

    def forward(self, embeds):
        logists = torch.log_softmax(self.layer(embeds), 1)
        return logists

- 初始化模型

In [14]:
graphSage = GraphSage(config['setting.num_layers'], features.size(1), config['setting.hidden_emb_size'], features, getattr(dataCenter, ds+'_adj_lists'), device, gcn=args.gcn, agg_func=args.agg_func)
graphSage = graphSage.to(device)

num_labels = len(set(getattr(dataCenter, ds + '_labels')))  # label的数量
classification = Classification(config['setting.hidden_emb_size'], num_labels).to(device)

### 正负采样函数

In [15]:
class UnsupervisedLoss(object):
    """docstring for UnsupervisedLoss"""
    def __init__(self, adj_lists, train_nodes, device):
        super(UnsupervisedLoss, self).__init__()
        self.Q = 10
        self.N_WALKS = 6
        self.WALK_LEN = 1
        self.N_WALK_LEN = 5
        self.MARGIN = 3
        self.adj_lists = adj_lists
        self.train_nodes = train_nodes
        self.device = device

        self.target_nodes = None
        self.positive_pairs = []
        self.negtive_pairs = []
        self.node_positive_pairs = {}
        self.node_negtive_pairs = {}
        self.unique_nodes_batch = []

    def get_loss_sage(self, embeddings, nodes):
        assert len(embeddings) == len(self.unique_nodes_batch)
        assert False not in [nodes[i]==self.unique_nodes_batch[i] for i in range(len(nodes))]
        node2index = {n:i for i,n in enumerate(self.unique_nodes_batch)}

        nodes_score = []
        assert len(self.node_positive_pairs) == len(self.node_negtive_pairs)
        for node in self.node_positive_pairs:
            pps = self.node_positive_pairs[node]
            nps = self.node_negtive_pairs[node]
            if len(pps) == 0 or len(nps) == 0:
                continue

            # Q * Exception(negative score)
            indexs = [list(x) for x in zip(*nps)]
            node_indexs = [node2index[x] for x in indexs[0]]
            neighb_indexs = [node2index[x] for x in indexs[1]]
            neg_score = F.cosine_similarity(embeddings[node_indexs], embeddings[neighb_indexs])
            neg_score = self.Q*torch.mean(torch.log(torch.sigmoid(-neg_score)), 0)
            # print(neg_score)

            # multiple positive score
            indexs = [list(x) for x in zip(*pps)]
            node_indexs = [node2index[x] for x in indexs[0]]
            neighb_indexs = [node2index[x] for x in indexs[1]]
            pos_score = F.cosine_similarity(embeddings[node_indexs], embeddings[neighb_indexs])
            pos_score = torch.log(torch.sigmoid(pos_score))
            # print(pos_score)

            nodes_score.append(torch.mean(- pos_score - neg_score).view(1,-1))
                
        loss = torch.mean(torch.cat(nodes_score, 0))
        
        return loss

    def get_loss_margin(self, embeddings, nodes):
        assert len(embeddings) == len(self.unique_nodes_batch)
        assert False not in [nodes[i]==self.unique_nodes_batch[i] for i in range(len(nodes))]
        node2index = {n:i for i,n in enumerate(self.unique_nodes_batch)}

        nodes_score = []
        assert len(self.node_positive_pairs) == len(self.node_negtive_pairs)
        for node in self.node_positive_pairs:
            pps = self.node_positive_pairs[node]
            nps = self.node_negtive_pairs[node]
            if len(pps) == 0 or len(nps) == 0:
                continue

            indexs = [list(x) for x in zip(*pps)]
            node_indexs = [node2index[x] for x in indexs[0]]
            neighb_indexs = [node2index[x] for x in indexs[1]]
            pos_score = F.cosine_similarity(embeddings[node_indexs], embeddings[neighb_indexs])
            pos_score, _ = torch.min(torch.log(torch.sigmoid(pos_score)), 0)

            indexs = [list(x) for x in zip(*nps)]
            node_indexs = [node2index[x] for x in indexs[0]]
            neighb_indexs = [node2index[x] for x in indexs[1]]
            neg_score = F.cosine_similarity(embeddings[node_indexs], embeddings[neighb_indexs])
            neg_score, _ = torch.max(torch.log(torch.sigmoid(neg_score)), 0)

            nodes_score.append(torch.max(torch.tensor(0.0).to(self.device), neg_score-pos_score+self.MARGIN).view(1,-1))
            # nodes_score.append((-pos_score - neg_score).view(1,-1))

        loss = torch.mean(torch.cat(nodes_score, 0),0)

        # loss = -torch.log(torch.sigmoid(pos_score))-4*torch.log(torch.sigmoid(-neg_score))
        
        return loss


    def extend_nodes(self, nodes, num_neg=6):
        self.positive_pairs = []
        self.node_positive_pairs = {}
        self.negtive_pairs = []
        self.node_negtive_pairs = {}

        self.target_nodes = nodes
        self.get_positive_nodes(nodes)
        # print(self.positive_pairs)
        self.get_negtive_nodes(nodes, num_neg)
        # print(self.negtive_pairs)
        self.unique_nodes_batch = list(set([i for x in self.positive_pairs for i in x]) | set([i for x in self.negtive_pairs for i in x]))
        assert set(self.target_nodes) < set(self.unique_nodes_batch)
        return self.unique_nodes_batch

    def get_positive_nodes(self, nodes):
        return self._run_random_walks(nodes)

    def get_negtive_nodes(self, nodes, num_neg):
        for node in nodes:
            neighbors = set([node])
            frontier = set([node])
            for i in range(self.N_WALK_LEN):
                current = set()
                for outer in frontier:
                    current |= self.adj_lists[int(outer)]
                frontier = current - neighbors
                neighbors |= current
            far_nodes = set(self.train_nodes) - neighbors
            neg_samples = random.sample(far_nodes, num_neg) if num_neg < len(far_nodes) else far_nodes
            self.negtive_pairs.extend([(node, neg_node) for neg_node in neg_samples])
            self.node_negtive_pairs[node] = [(node, neg_node) for neg_node in neg_samples]
        return self.negtive_pairs

    def _run_random_walks(self, nodes):
        for node in nodes:
            if len(self.adj_lists[int(node)]) == 0:
                continue
            cur_pairs = []
            for i in range(self.N_WALKS):
                curr_node = node
                for j in range(self.WALK_LEN):
                    neighs = self.adj_lists[int(curr_node)]
                    next_node = random.choice(list(neighs))
                    # self co-occurrences are useless
                    if next_node != node and next_node in self.train_nodes:
                        self.positive_pairs.append((node,next_node))
                        cur_pairs.append((node,next_node))
                    curr_node = next_node
            
            self.node_positive_pairs[node] = cur_pairs
        return self.positive_pairs

- 初始化正负采样

In [16]:
unsupervised_loss = UnsupervisedLoss(getattr(dataCenter, ds + '_adj_lists'), getattr(dataCenter, ds + '_train'), device)

### GraphSage层特征编码

In [17]:
def get_gnn_embeddings(gnn_model, dataCenter, ds):
    print('Loading embeddings from trained GraphSAGE model.')
    features = np.zeros((len(getattr(dataCenter, ds + '_labels')), gnn_model.out_size))
    nodes = np.arange(len(getattr(dataCenter, ds+'_labels'))).tolist()
    b_sz = 500
    batches = math.ceil(len(nodes) / b_sz)
    embs = []
    for index in range(batches):
        nodes_batch = nodes[index * b_sz: (index + 1) * b_sz]
        embs_batch = gnn_model(nodes_batch)
        assert len(embs_batch) == len(nodes_batch)
        embs.append(embs_batch)
        # if ((index+1)*b_sz) % 10000 == 0:
        #     print(f'Dealed Nodes [{(index+1)*b_sz}/{len(nodes)}]')

    assert len(embs) == batches
    embs = torch.cat(embs, 0)
    assert len(embs) == len(nodes)
    print('Embeddings loaded.')
    return embs.detach()

In [18]:
# 测试embedding效果
features = get_gnn_embeddings(graphSage, dataCenter, ds)
print(features)

Loading embeddings from trained GraphSAGE model.
Embeddings loaded.
tensor([[0.0052, 0.0000, 0.1288,  ..., 0.1129, 0.0452, 0.0432],
        [0.0000, 0.0000, 0.0713,  ..., 0.0618, 0.0213, 0.0733],
        [0.0000, 0.0000, 0.0330,  ..., 0.0000, 0.0866, 0.0316],
        ...,
        [0.0000, 0.0000, 0.3030,  ..., 0.0000, 0.0195, 0.1570],
        [0.0000, 0.0000, 0.2029,  ..., 0.0070, 0.0190, 0.0342],
        [0.0151, 0.0000, 0.1297,  ..., 0.0725, 0.0000, 0.0512]],
       device='cuda:0')


### 一个Epoch的模型训练，生成：
- graphSage模型：对节点进行编码
- classification模型：对节点进行分类

In [19]:
def apply_model(dataCenter, ds, graphSage, classification, unsupervised_loss, b_sz, unsup_loss, device, learn_method):
    test_nodes = getattr(dataCenter, ds + '_test')
    val_nodes = getattr(dataCenter, ds + '_val')
    train_nodes = getattr(dataCenter, ds + '_train')
    labels = getattr(dataCenter, ds + '_labels')

    if unsup_loss == 'margin':
        num_neg = 6
    elif unsup_loss == 'normal':
        num_neg = 100
    else:
        print("unsup_loss can be only 'margin' or 'normal'.")
        sys.exit(1)

    train_nodes = shuffle(train_nodes)

    models = [graphSage, classification]
    params = []
    for model in models:
        for param in model.parameters():
            if param.requires_grad:
                params.append(param)

    optimizer = torch.optim.SGD(params, lr=0.7)
    optimizer.zero_grad()
    for model in models:
        model.zero_grad()

    batches = math.ceil(len(train_nodes) / b_sz)

    visited_nodes = set()
    for index in range(batches):
        nodes_batch = train_nodes[index * b_sz: (index + 1) * b_sz]  # batch训练的节点
        
        # print(nodes_batch.shape)
        # print(nodes_batch)

        # extend nodes batch for unspervised learning
        # no conflicts with supervised learning
        nodes_batch = np.asarray(list(unsupervised_loss.extend_nodes(nodes_batch, num_neg=num_neg)))
        visited_nodes |= set(nodes_batch)

        # get ground-truth for the nodes batch
        labels_batch = labels[nodes_batch]

        # feed nodes batch to the graphSAGE
        # returning the nodes embeddings。 得到GraphSAGE后的ebmedding向量
        embs_batch = graphSage(nodes_batch)  # 跳到models的GraphSge

        if learn_method == 'sup':
            # superivsed learning
            logists = classification(embs_batch)
            loss_sup = -torch.sum(logists[range(logists.size(0)), labels_batch], 0)
            loss_sup /= len(nodes_batch)
            loss = loss_sup
        elif learn_method == 'plus_unsup':
            # superivsed learning
            logists = classification(embs_batch)
            loss_sup = -torch.sum(logists[range(logists.size(0)), labels_batch], 0)
            loss_sup /= len(nodes_batch)
            # unsuperivsed learning
            if unsup_loss == 'margin':
                loss_net = unsupervised_loss.get_loss_margin(embs_batch, nodes_batch)
            elif unsup_loss == 'normal':
                loss_net = unsupervised_loss.get_loss_sage(embs_batch, nodes_batch)
            loss = loss_sup + loss_net
        else:
            if unsup_loss == 'margin':
                loss_net = unsupervised_loss.get_loss_margin(embs_batch, nodes_batch)
            elif unsup_loss == 'normal':
                loss_net = unsupervised_loss.get_loss_sage(embs_batch, nodes_batch)
            loss = loss_net

        print('Step [{}/{}], Loss: {:.4f}, Dealed Nodes [{}/{}] '.format(index+1, batches, loss.item(), len(visited_nodes), len(train_nodes)))
        loss.backward()
        for model in models:
            nn.utils.clip_grad_norm_(model.parameters(), 5)  # 梯度的二范数和不超过5（平方和开根号）
        optimizer.step()

        optimizer.zero_grad()
        for model in models:
            model.zero_grad()

    return graphSage, classification

### 模型测试

In [20]:
def evaluate(dataCenter, ds, graphSage, classification, device, max_vali_f1, name, cur_epoch):
    test_nodes = getattr(dataCenter, ds+'_test')
    val_nodes = getattr(dataCenter, ds+'_val')
    labels = getattr(dataCenter, ds+'_labels')

    models = [graphSage, classification]

    params = []
    for model in models:
        for param in model.parameters():
            if param.requires_grad:
                param.requires_grad = False
                params.append(param)

    embs = graphSage(val_nodes)
    logists = classification(embs)
    _, predicts = torch.max(logists, 1)
    labels_val = labels[val_nodes]
    assert len(labels_val) == len(predicts)
    comps = zip(labels_val, predicts.data)

    vali_f1 = f1_score(labels_val, predicts.cpu().data, average="micro")
    print("Validation F1:", vali_f1)

    if vali_f1 > max_vali_f1:
        max_vali_f1 = vali_f1
        embs = graphSage(test_nodes)
        logists = classification(embs)
        _, predicts = torch.max(logists, 1)
        labels_test = labels[test_nodes]
        assert len(labels_test) == len(predicts)
        comps = zip(labels_test, predicts.data)

        test_f1 = f1_score(labels_test, predicts.cpu().data, average="micro")
        print("Test F1:", test_f1)

        for param in params:
            param.requires_grad = True

        torch.save(models, './model_best_{}_ep{}_{:.4f}.torch'.format(name, cur_epoch, test_f1))

    for param in params:
        param.requires_grad = True

    return max_vali_f1

### 多个Epoch进行训练

In [21]:
for epoch in range(args.epochs):
    print('----------------------EPOCH %d-----------------------' % epoch)
    graphSage, classification = apply_model(dataCenter, ds, graphSage, classification, unsupervised_loss, args.b_sz, args.unsup_loss, device, args.learn_method)
    
    # 模型测试
    if (epoch + 1) % 2 == 0 and args.learn_method == 'unsup':
        classification, args.max_vali_f1 = train_classification(dataCenter, graphSage, classification, ds, device, args.max_vali_f1, args.name)
    if args.learn_method != 'unsup':
        args.max_vali_f1 = evaluate(dataCenter, ds, graphSage, classification, device, args.max_vali_f1, args.name, epoch)

----------------------EPOCH 0-----------------------
Step [1/68], Loss: 1.9446, Dealed Nodes [1018/1355] 
Step [2/68], Loss: 1.8598, Dealed Nodes [1263/1355] 
Step [3/68], Loss: 1.7914, Dealed Nodes [1325/1355] 
Step [4/68], Loss: 1.7095, Dealed Nodes [1344/1355] 
Step [5/68], Loss: 1.6209, Dealed Nodes [1353/1355] 
Step [6/68], Loss: 1.5517, Dealed Nodes [1355/1355] 
Step [7/68], Loss: 1.4243, Dealed Nodes [1355/1355] 
Step [8/68], Loss: 1.3042, Dealed Nodes [1355/1355] 
Step [9/68], Loss: 1.1833, Dealed Nodes [1355/1355] 
Step [10/68], Loss: 1.0458, Dealed Nodes [1355/1355] 
Step [11/68], Loss: 0.9221, Dealed Nodes [1355/1355] 
Step [12/68], Loss: 0.8258, Dealed Nodes [1355/1355] 
Step [13/68], Loss: 0.7263, Dealed Nodes [1355/1355] 
Step [14/68], Loss: 0.6574, Dealed Nodes [1355/1355] 
Step [15/68], Loss: 0.6323, Dealed Nodes [1355/1355] 
Step [16/68], Loss: 0.8439, Dealed Nodes [1355/1355] 
Step [17/68], Loss: 2.1830, Dealed Nodes [1355/1355] 
Step [18/68], Loss: 1.2149, Dealed Nod

c:\users\sss\appdata\local\programs\python\python36\lib\site-packages\torch\serialization.py:250: UserWarning: Couldn't retrieve source code for container of type GraphSage. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
c:\users\sss\appdata\local\programs\python\python36\lib\site-packages\torch\serialization.py:250: UserWarning: Couldn't retrieve source code for container of type SageLayer. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
c:\users\sss\appdata\local\programs\python\python36\lib\site-packages\torch\serialization.py:250: UserWarning: Couldn't retrieve source code for container of type Classification. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


----------------------EPOCH 1-----------------------
Step [1/68], Loss: 0.0433, Dealed Nodes [1039/1355] 
Step [2/68], Loss: 0.0375, Dealed Nodes [1262/1355] 
Step [3/68], Loss: 0.0345, Dealed Nodes [1323/1355] 
Step [4/68], Loss: 0.0379, Dealed Nodes [1338/1355] 
Step [5/68], Loss: 0.0325, Dealed Nodes [1348/1355] 
Step [6/68], Loss: 0.0335, Dealed Nodes [1350/1355] 
Step [7/68], Loss: 0.0334, Dealed Nodes [1351/1355] 
Step [8/68], Loss: 0.0323, Dealed Nodes [1354/1355] 
Step [9/68], Loss: 0.0310, Dealed Nodes [1355/1355] 
Step [10/68], Loss: 0.0272, Dealed Nodes [1355/1355] 
Step [11/68], Loss: 0.0243, Dealed Nodes [1355/1355] 
Step [12/68], Loss: 0.0278, Dealed Nodes [1355/1355] 
Step [13/68], Loss: 0.0256, Dealed Nodes [1355/1355] 
Step [14/68], Loss: 0.0274, Dealed Nodes [1355/1355] 
Step [15/68], Loss: 0.0244, Dealed Nodes [1355/1355] 
Step [16/68], Loss: 0.0262, Dealed Nodes [1355/1355] 
Step [17/68], Loss: 0.0236, Dealed Nodes [1355/1355] 
Step [18/68], Loss: 0.0252, Dealed Nod